##People detection
The next function is meant to detect the people in our input video

In [ ]:
# base path to YOLO directory
MODEL_PATH = "yolo-coco"

# weak detections in video by minimum probability and threshold when using maxima suppression
minimun_conf = 0.3
thresh = 0.3

# if we want to use GPU, cuda or something like that
USE_GPU = False

# the team is using pixels to determine if to people are to close together, the minimum safe distance is 
MIN_DISTANCE = 50

# libraries
import numpy as np
import cv2

# funtion to detect people
def detect_people(frame, net_dark, layer_names, personIdx=0):

	# dimention of the frames
	(H, W) = frame.shape[:2]
	resultados = []

	# blobs can store images, binary data
	blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
	net_dark.setInput(blob)
	layerOutputs = net_dark.forward(layer_names)

  # making list of boxes, their centroids and condidences
	boxes = []
	centroids = []
	confidences = []

	# loop over each of the layer outputs
	for output in layerOutputs:
		# loop over each of the detections
		for detection in output:
      # getting to know the class ID and the scores
			scores = detection[5:]
			classID = np.argmax(scores)
			confidence = scores[classID]

      # ensuring that we are detecting a persona and having the minimum confidence
			if classID == personIdx and confidence > minimun_conf:
				
        # scaling the boxes that represent the detection of the people
				box = detection[0:4] * np.array([W, H, W, H])
				(centerX, centerY, width, height) = box.astype("int")

        # defining top and corners of the box
				x = int(centerX - (width / 2))
				y = int(centerY - (height / 2))

        # update list of boxes values, (centroids)
				boxes.append([x, y, int(width), int(height)])
				centroids.append((centerX, centerY))
				confidences.append(float(confidence))

  # applying non maxima suppression for weak overlapping boxes
	idxs = cv2.dnn.NMSBoxes(boxes, confidences, minimun_conf, thresh)

	# there must be at least one detection
	if len(idxs) > 0:
		# loop over the indexes we are keeping
		for i in idxs.flatten():
			# extract the bounding box coordinates
			(x, y) = (boxes[i][0], boxes[i][1])
			(w, h) = (boxes[i][2], boxes[i][3])

			# update list to of the person prediction probability
			r = (confidences[i], (x, y, x + w, y + h), centroids[i])
			resultados.append(r)

	# return the list of results
	return resultados

##Testing on frame
Testing the people detector for one frame

In [ ]:
# we are using this resources
# https://pjreddie.com/darknet/yolo/

#wget https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg
configPath = os.path.sep.join([MODEL_PATH, "yolov3.cfg"])

#wget https://pjreddie.com/media/files/yolov3.weights
weightsPath = os.path.sep.join([MODEL_PATH, "yolov3.weights"])

net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

resultados = detect_people(frame, net, ln, personIdx=LABELS.index("person"))